In [ ]:
from engine import GameEngine
from core_components.display import Display
from core_components.loop import GameLoop
from core_components.store import GameStore

game = GameEngine(display=Display(), loop=GameLoop(), store=GameStore())

In [ ]:
game.state, game.display.state, game.loop.state, game.store.state

In [ ]:
game.start()
game.state, game.display.state, game.loop.state, game.store.state

In [ ]:
game.play()
game.state, game.display.state, game.loop.state, game.store.state

In [ ]:
game.pause()
game.state, game.display.state, game.loop.state, game.store.state

In [ ]:
game.play()
game.state, game.display.state, game.loop.state, game.store.state

In [ ]:
game.reset()
game.state, game.display.state, game.loop.state, game.store.state

In [ ]:
game.stop()
game.state, game.display.state, game.loop.state, game.store.state


In [ ]:
test

In [ ]:
from type_protocols import StatefulObject
from core_components.loops import BaseLoopHandler, BaseGameEvent
from core_components.loop import GameLoop
from core_components.store import GameStore
import time
class NonEvent(BaseGameEvent):
    
    def trigger(self) -> None:
        print("Nothing happened.")
        super().trigger()

loop = GameLoop()
event = NonEvent()
event.store = GameStore()
event.transformer = loop.handler

loop.start()
event.trigger()
time.sleep(1)

loop.pause()
event.trigger()

loop.start()
event.trigger()
time.sleep(2)

In [1]:
from core_components.store import GameStore

a = GameStore()
a.start()


Game Store is started.


c:\Users\jason\workspaces\repos\jrl\src\core_components\maps\tiles\base.py:469: UserWarning: Cannot align corners without 'center', 'width', and 'height' attributes set.
  warn("Cannot align corners without 'center', 'width', and 'height' attributes set.", UserWarning)
c:\Users\jason\workspaces\repos\jrl\src\core_components\maps\tiles\base.py:469: UserWarning: Cannot align corners without 'center', 'width', and 'height' attributes set.
  warn("Cannot align corners without 'center', 'width', and 'height' attributes set.", UserWarning)
c:\Users\jason\workspaces\repos\jrl\src\core_components\maps\tiles\base.py:395: UserWarning: Height 1 is less than minimum dimension size 3. Setting height to minimum.
  warn(f"Height {value} is less than minimum dimension size {self._min_dimension_size}. Setting height to minimum.", UserWarning)
c:\Users\jason\workspaces\repos\jrl\src\core_components\maps\tiles\base.py:380: UserWarning: Width 1 is less than minimum dimension size 3. Setting width to minim

True

In [3]:
a.player

In [ ]:
from transitions import Machine

class Dummy:    
    def __init__(self):
        states = ['started', 'stopped']
        transitions =[
            {'trigger': 'start', 'source': 'stopped', 'dest': 'started'},
            {'trigger': 'stop', 'source': 'started', 'dest': 'stopped'}
            ]
        self.x = Machine(model=self, states=states, transitions=transitions, initial='stopped')

a = Dummy()
a.state

In [ ]:
a.state

In [ ]:
event = NonEvent()
event.store = GameStore()
event.transformer = loop.handler

loop.handler.events.put(event)

In [ ]:
event.trigger()

In [ ]:
loop.pause()
event.trigger()

In [ ]:
loop.start()

In [ ]:
loop.